In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder


In [10]:
df = pd.read_csv('train_split.csv')

    # 타겟 이진화
if '임신 성공 여부' in df.columns:
    df['임신 성공 여부'] = df['임신 성공 여부'].apply(lambda x: 1 if x > 0 else 0)

    # 횟수형 문자열 → 정수
count_columns = [
    "총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수",
    "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수",
    "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수"
]

def convert_count(value):
    if isinstance(value, str) and "회 이상" in value:
        return 6
    try:
        return int(str(value).replace("회", ""))
    except:
        return 0

for col in count_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).apply(convert_count)

    # 시술 당시 나이
age_mapping = {
        "만18-34세": 25,
        "만35-37세": 36,
        "만38-39세": 38,
        "만40-42세": 41,
        "만43-44세": 43,
        "만45-50세": 44,
        "알 수 없음": np.nan}

if "시술 당시 나이" in df.columns:
    df["시술 당시 나이 (변환)"] = df["시술 당시 나이"].map(age_mapping)
    df["시술 당시 나이 (변환)"] = df["시술 당시 나이 (변환)"].fillna(df["시술 당시 나이 (변환)"].mean())

    # 시술 코드 multi-hot 처리
# procedure_col = df["특정 시술 유형"].fillna('Unknown')
# procedure_tokens = [
#         'IVF', 'ICSI', 'IUI', 'ICI', 'GIFT', 'FER', 'Generic DI',
#         'IVI', 'BLASTOCYST', 'AH', 'Unknown'
# ]
#
# for token in procedure_tokens:
#     df[f'is_{token.lower().replace(" ", "_")}'] = procedure_col.apply(
#         lambda x: int(token in str(x).replace('/', ':').split(':'))
#         )
#
# df = df.drop(columns=["특정 시술 유형"])

# 문자열 피처 → ordinal encoding
# string_cols = df.select_dtypes(include='object').columns.difference(['특정 시술 유형'])
# for col in string_cols:
#     df[col] = df[col].fillna("NaN").astype(str)
# if len(string_cols) > 0:
#     df[string_cols] = OrdinalEncoder().fit_transform(df[string_cols])

    # 수치형 피처 → 중앙값 대체
numeric_cols = df.select_dtypes(include='number').columns.difference(['임신 성공 여부'])
for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())

    # 나이 결측 보간 (평균)
if "시술 당시 나이 (변환)" in df.columns:
    df["시술 당시 나이 (변환)"] = df["시술 당시 나이 (변환)"].fillna(df["시술 당시 나이 (변환)"].mean())

for col in df.select_dtypes(include='number').columns:
    unique_vals = df[col].dropna().unique()
    if set(unique_vals).issubset({0, 1}):  # 이진 변수일 경우
        df[col] = df[col].astype(str)

print(df.shape[1])
print(df.dtypes)

69
시술 시기 코드                  object
시술 당시 나이                  object
임신 시도 또는 마지막 임신 경과 연수    float64
시술 유형                     object
특정 시술 유형                  object
                          ...   
난자 혼합 경과일                float64
배아 이식 경과일                float64
배아 해동 경과일                float64
임신 성공 여부                  object
시술 당시 나이 (변환)            float64
Length: 69, dtype: object


In [11]:
cat_cols = df.select_dtypes(include='object').columns.tolist()
num_cols = df.select_dtypes(include='number').drop('임신 성공 여부', errors='ignore').columns.tolist()
cat_dims = [df[col].nunique() for col in cat_cols]

print(cat_cols, cat_dims, num_cols, len(num_cols))

['시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 채취 경과일', '난자 해동 경과일', '임신 성공 여부'] [7, 7, 2, 24, 2, 4, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 13, 3, 4, 5, 7, 2, 2, 2, 2, 1, 1, 1, 2, 2] ['임신 시도 또는 마지막 임신 경과 연수', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 